In [ ]:
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
quiet_library(viridis)
quiet_library(harmony)
quiet_library(Nebulosa)

In [ ]:
quiet_library(ArchR)

In [ ]:
addArchRThreads(32)
addArchRGenome("hg38")

# Load Data

In [ ]:
sp_merge <- readRDS(file = '../../Seurat_Objects/sp_merge_gating_celltype_updated.rds')

In [ ]:
sp_merge <- SetIdent(sp_merge, value = 'gating_celltype')
cd4_mem <- subset(sp_merge, idents = c('CD4 CM','CD4 EM1','CD4 EM2','CD4 TEMRA','Treg'))

In [ ]:
cd4_mem$pure_celltype <- cd4_mem$gating_celltype

In [ ]:
cd4na <- readRDS(file = 'cd4_na_labeled.rds')

In [ ]:
cd4_merge <- merge(cd4na, cd4_mem)

In [ ]:
rm(sp_merge)
rm(cd4_mem)
rm(cd4na)

# ATAC

In [ ]:
proj <- loadArchRProject(path = '../../PedSen_ATAC/')
proj

In [ ]:
bcs <- colnames(cd4_merge[['RNA']])
head(bcs)

In [ ]:
idx_new <- which(proj$cellNames_clean %in% bcs)
head(idx_new)

In [ ]:
cells_subset <- proj$cellNames[idx_new]
proj_subset <- proj[cells_subset,]
proj_subset

### Add Pure Celltypes

In [ ]:
pure_clusters <- as.data.frame(cd4_merge$pure_celltype)
rownames(pure_clusters) <- paste(cd4_merge$batch_id, "-P1_",cd4_merge$pbmc_sample_id,"#",rownames(pure_clusters), sep = "")
colnames(pure_clusters) <- 'pure_clusters'
pure_clusters[] <- lapply(pure_clusters, as.character)
head(pure_clusters)

In [ ]:
proj_subset <- addCellColData(proj_subset, data = pure_clusters$pure_clusters, name = 'pure_celltype', cells = rownames(pure_clusters), force = TRUE)

# ATAC Clustering

In [ ]:
proj_subset <- addIterativeLSI(proj_subset, name = 'IterativeLSI', force = TRUE)

In [ ]:
proj_subset <- addHarmony(proj_subset, reducedDims = 'IterativeLSI', groupBy = 'batch_id', force = TRUE)

In [ ]:
#proj_subset <- addClusters(proj_subset, reducedDims = 'IterativeLSI', name = 'Clusters', force = TRUE, resolution = 0.2)
proj_subset <- addClusters(proj_subset, reducedDims = 'Harmony', name = 'Clusters', force = TRUE, resolution = 0.2)

In [ ]:
#proj_subset <- addUMAP(proj_subset, reducedDims = 'IterativeLSI', name = 'UMAP', force = TRUE)
proj_subset <- addUMAP(proj_subset, reducedDims = 'Harmony', name = 'UMAP', force = TRUE)

In [ ]:
options(repr.plocd8_naidth = 8, repr.plot.height = 8)
plotEmbedding(ArchRProj = proj_subset, colorBy = "cellColData", name = "Clusters", embedding = "UMAP")
plotEmbedding(ArchRProj = proj_subset, colorBy = "cellColData", name = "pediatric_senior", embedding = "UMAP")
# plotEmbedding(ArchRProj = proj_subset, colorBy = "cellColData", name = "wnn_celltype", embedding = "UMAP")
plotEmbedding(ArchRProj = proj_subset, colorBy = "cellColData", name = "pure_celltype", embedding = "UMAP")

# Peak Analysis

In [ ]:
proj_subset <- addGroupCoverages(ArchRProj = proj_subset, groupBy = "pure_celltype", maxReplicates = 16, maxCells = 2000, force = TRUE)

In [ ]:
pathToMacs2 <- findMacs2()

In [ ]:
proj_subset <- addReproduciblePeakSet(
                        ArchRProj = proj_subset, 
                        groupBy = "pure_celltype", 
                        pathToMacs2 = pathToMacs2, force = TRUE
                            )

In [ ]:
proj_subset <- addPeakMatrix(proj_subset)

# Peak Heatmap - Ext Fig3B

In [ ]:
table(proj_subset$pure_celltype)

In [ ]:
markersPeaks_pure <- getMarkerFeatures(maxCells = 1000,
    ArchRProj = proj_subset,
    useMatrix = "PeakMatrix", 
    groupBy = "pure_celltype",
  bias = c("TSSEnrichment", "log10(nFrags)"),
  testMethod = "wilcoxon"
)

In [ ]:
heatmapPeaks_pure <- markerHeatmap(
  seMarker = markersPeaks_pure, 
  cutOff = "FDR <= 0.1 & Log2FC >= 0.5",
  transpose = FALSE
)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 12)
draw(heatmapPeaks_pure, heatmap_legend_side = "right", annotation_legend_side = "bot")

In [ ]:
# Open a pdf file
pdf("plots/cd4all_peaks_atac_heatmap.pdf", width = 8, height = 12) 
# 2. Create a plot
draw(heatmapPeaks_pure, heatmap_legend_side = "right", annotation_legend_side = "bot")
# Close the pdf file
dev.off() 